# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

### Questões

1. Quais os grupos possuem mais aderência as campanhas?
2. Qual o tipo de campanha que possui mais aderência?
3. Detalhe entre cada grupo, quais são as campanhas mais aderentes.
4. Qual a quantidade de ofertas que os usuários não viram?
5. Qual a quantidade de ofertas não lidas pelos usuários, porém validadas na compra?
6. Qual a quantidade de ofertas vistas pelos usuários, porém não validas na compra?

***To-do:***
* Crie ranges de idade para cada usuário

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

In [447]:
import pandas as pd
import numpy as np
import math
import json
#% matplotlib inline

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [448]:
#rename columns
portfolio = portfolio.rename(columns={'id': 'offer_id'})
#Show df to get an idea of the data
portfolio.sort_values('offer_type', ascending=False).head(10)

,reward,channels,difficulty,duration,offer_type,offer_id
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d


In [449]:
#converting the 'became_member_on' column into datetime
profile['became_member_on'] = pd.to_datetime(profile['became_member_on'].astype(str), format='%Y%m%d')
#rename columns
profile = profile.rename(columns={'id': 'user_id'})
#Show df to get an idea of the data
profile.head()

,gender,age,user_id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,2017-08-04,NaN


In [450]:
#normalizing dict row into columns
transcript = transcript.join(pd.json_normalize(transcript.value))

#drop value column
transcript.drop(columns=['value'], inplace=True)

#rename columns
transcript = transcript.rename(columns={'person': 'user_id', 'offer id': 'offer_id_received', 'offer_id': 'offer_id_completed'})

In [451]:
transcript.head()

,user_id,event,time,offer_id_received,amount,offer_id_completed,reward
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,NaN
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,NaN
2,e2127556f4f64592b11af22de27a7932,offer received,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN,NaN
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN,NaN


In [452]:
transcript['event'].value_counts()

transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: event, dtype: int64

## OFERTAS COMPLETAS

In [453]:
#df with all completed offers
offers_completed = transcript[transcript['event'] == 'offer completed']
#rename column name
offers_completed = offers_completed.rename(columns={'offer_id_completed': 'offer_id'})
#drop value column
offers_completed.drop(columns=['offer_id_received', 'amount', 'reward'], inplace=True)

In [454]:
offers_completed = offers_completed.merge(profile, how='left', on='user_id')
offers_completed = offers_completed.merge(portfolio, how='left', on='offer_id')

In [455]:
offers_completed.head()

,user_id,event,time,offer_id,gender,age,became_member_on,income,reward,channels,difficulty,duration,offer_type
0,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer completed,0,2906b810c7d4411798c6938adc9daaa5,M,42,2016-01-17,96000.0,2,"[web, email, mobile]",10,7,discount
1,fe97aa22dd3e48c8b143116a8403dd52,offer completed,0,fafdcd668e3743c1bb461111dcafc2a4,F,39,2017-12-17,67000.0,2,"[web, email, mobile, social]",10,10,discount
2,629fc02d56414d91bca360decdfa9288,offer completed,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,M,52,2018-06-05,72000.0,5,"[web, email, mobile]",5,7,bogo
3,676506bad68e4161b9bbaffeb039626b,offer completed,0,ae264e3637204a6fb9bb56bc8210ddfd,M,37,2017-05-15,92000.0,10,"[email, mobile, social]",10,7,bogo
4,8f7dd3b2afe14c078eb4f6e6fe4ba97d,offer completed,0,4d5c57ea9a6940dd891ad53e9dbe8da0,M,48,2015-09-03,62000.0,10,"[web, email, mobile, social]",10,5,bogo


### 1. Quais os grupos que possuem mais aderência as campanhas?

In [456]:
#grupo de usuários que mais completaram as campanhas
offers_completed.groupby('gender').mean()

,time,age,income,reward,difficulty,duration
gender,,,,,,
F,396.899399,57.972605,73313.303612,5.090586,9.413452,7.288622
M,402.753917,53.819507,65911.150249,4.812401,9.216628,7.346836
O,380.383234,55.499002,65169.660679,5.015968,9.449102,7.325349


### 2. Qual o tipo de campanha que possui mais aderência?

In [457]:
#grupo de usuários que mais completaram as campanhas
offers_completed.groupby('offer_type')['user_id'].count()/offers_completed.shape[0]*100

offer_type
bogo        46.663093
discount    53.336907
Name: user_id, dtype: float64

### 3. Detalhe entre cada grupo, quais são as campanhas mais aderentes.

In [458]:
offers_completed.groupby(['gender', 'offer_type'])['user_id'].count()/offers_completed.shape[0]*100

gender  offer_type
F       bogo          22.338366
        discount      23.752941
M       bogo          22.371125
        discount      26.665475
O       bogo           0.729623
        discount       0.762381
Name: user_id, dtype: float64

## TRANSAÇÕES

In [459]:
#df with all transactions
transactions = transcript[transcript['event'] == 'transaction']
#drop value column
transactions.drop(columns=['offer_id_received', 'offer_id_completed', 'reward'], inplace=True)

In [460]:
transactions = transactions.merge(profile, how='left', on='user_id')

In [461]:
transactions.head()

,user_id,event,time,amount,gender,age,became_member_on,income
0,02c083884c7d45b39cc68e1314fec56c,transaction,0,0.83,F,20,2016-07-11,30000.0
1,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,0,34.56,M,42,2016-01-17,96000.0
2,54890f68699049c2a04d415abc25e717,transaction,0,13.23,M,36,2017-12-28,56000.0
3,b2f1cd155b864803ad8334cdf13c4bd2,transaction,0,19.51,F,55,2017-10-16,94000.0
4,fe97aa22dd3e48c8b143116a8403dd52,transaction,0,18.97,F,39,2017-12-17,67000.0


## OFERTAS VISUALIZADAS

In [462]:
#df with all offers viewed
offers_viewed = transcript[transcript['event'] == 'offer viewed']
#rename column name
offers_viewed = offers_viewed.rename(columns={'offer_id_received': 'offer_id'})
#drop value column
offers_viewed.drop(columns=['reward', 'amount', 'offer_id_completed'], inplace=True)

In [463]:
offers_viewed = offers_viewed.merge(profile, how='left', on='user_id')
offers_viewed = offers_viewed.merge(portfolio, how='left', on='offer_id')

In [464]:
offers_viewed.head()

,user_id,event,time,offer_id,gender,age,became_member_on,income,reward,channels,difficulty,duration,offer_type
0,389bc3fa690240e798340f5a15918d5c,offer viewed,0,f19421c1d4aa40978ebb69ca19b0e20d,M,65,2018-02-09,53000.0,5,"[web, email, mobile, social]",5,5,bogo
1,d1ede868e29245ea91818a903fec04c6,offer viewed,0,5a8bc65990b245e5a138643cd4eb9837,O,53,2017-09-16,52000.0,0,"[email, mobile, social]",0,3,informational
2,102e9454054946fda62242d2e176fdce,offer viewed,0,4d5c57ea9a6940dd891ad53e9dbe8da0,F,69,2016-08-14,57000.0,10,"[web, email, mobile, social]",10,5,bogo
3,02c083884c7d45b39cc68e1314fec56c,offer viewed,0,ae264e3637204a6fb9bb56bc8210ddfd,F,20,2016-07-11,30000.0,10,"[email, mobile, social]",10,7,bogo
4,be8a5d1981a2458d90b255ddc7e0d174,offer viewed,0,5a8bc65990b245e5a138643cd4eb9837,M,39,2014-05-27,51000.0,0,"[email, mobile, social]",0,3,informational


In [472]:
#Tipo de campanha mais visualizada
offers_viewed.groupby('offer_type')['user_id'].count()/offers_viewed.shape[0]*100

offer_type
bogo             44.086618
discount         37.150282
informational    18.763101
Name: user_id, dtype: float64

## OFERTAS RECEBIDAS

In [465]:
#df with all offers received
offers_received = transcript[transcript['event'] == 'offer received']
#rename column name
offers_received = offers_received.rename(columns={'offer_id_received': 'offer_id'})
#drop value column
offers_received.drop(columns=['offer_id_completed', 'reward', 'amount'], inplace=True)

In [466]:
offers_received = offers_received.merge(profile, how='left', on='user_id')
offers_received = offers_received.merge(portfolio, how='left', on='offer_id')

In [467]:
offers_received.head()

,user_id,event,time,offer_id,gender,age,became_member_on,income,reward,channels,difficulty,duration,offer_type
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,2017-05-09,100000.0,5,"[web, email, mobile]",5,7,bogo
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,None,118,2017-08-04,NaN,5,"[web, email]",20,10,discount
2,e2127556f4f64592b11af22de27a7932,offer received,0,2906b810c7d4411798c6938adc9daaa5,M,68,2018-04-26,70000.0,2,"[web, email, mobile]",10,7,discount
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,fafdcd668e3743c1bb461111dcafc2a4,None,118,2017-09-25,NaN,2,"[web, email, mobile, social]",10,10,discount
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,4d5c57ea9a6940dd891ad53e9dbe8da0,None,118,2017-10-02,NaN,10,"[web, email, mobile, social]",10,5,bogo


In [479]:
#Tipo de campanha mais recebida
offers_received.groupby('offer_type')['user_id'].count()

offer_type
bogo             30499
discount         30543
informational    15235
Name: user_id, dtype: int64

## APENAS ANUNCIO

In [486]:
#df with all offers received
anuncio = transcript[transcript['event'] == 'offer received']
#rename column name
anuncio = offers_received.rename(columns={'offer_id_received': 'offer_id'})
#drop value column
#anuncio.drop(columns=['offer_id_completed', 'reward', 'amount'], inplace=True)

In [491]:
anuncio = anuncio[anuncio['offer_type'] == 'informational']
apenas_promocao = offers_received[offers_received['offer_type'] != 'informational']

In [496]:
apenas_promocao.shape

(61042, 13)

In [492]:
a = anuncio['user_id'].unique().tolist() #lista de usuários que receberam anuncio
b = apenas_promocao['user_id'].unique().tolist() #lista de usuários que receberam promocao

In [500]:
c = np.setdiff1d(a, b) #lista de usuários que receberam anuncio e não receberam promocao
len(c)

66